### STEP: 1

Import necessary libraries for **Predicting Future Stock Prices**

In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import hvplot.pandas
import requests

import warnings
warnings.filterwarnings('ignore')

# Libraries to Extract Stock Price Historical Data 
# from Yahoo! Finance using Tickers from Wikipedia
import yfinance as yf
from pandas.tseries.offsets import DateOffset
from datetime import datetime, timedelta
from pandas.tseries.holiday import USFederalHolidayCalendar

# Library for Technical Analysis
import pandas_ta as ta

# Libraries for ML Analysis using Linear Models
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import MultiTaskElasticNet
from sklearn.linear_model import LassoLars
from sklearn.linear_model import BayesianRidge

scaler = StandardScaler()
today = datetime.today().strftime('%Y-%m-%d')

### STEP: 2

Defining various functions used in the code for 
* extracting data, 
* adding technical indicators as features,
* predicting **Close** price and **VWAP** volume-weighted average price,
* using linear models for predicting with best model output.

Best model is selected based on minimum MAPE & RMSE

In [10]:
# Extracting Stock Price Historical Data from Yahoo! Finance using Tickers from Wikipedia
        
def extract_stock_data(stock_name, start_date):
    row_index = ticker_df.query(f'Security == "{stock_name}"').index[0]
    stock_ticker = ticker_list[row_index]
    OHLCV_df = yf.download(stock_ticker,start_date,today)
    OHLCV_df.index = pd.to_datetime(OHLCV_df.index)
    print(f"{stock_ticker}:{stock_name}")
    #display(OHLCV_df.head(3))
    #display(OHLCV_df.tail(3))
    return(OHLCV_df)

# Feature Engineering _ additionals features for ML Analysis

def technical_indicators():
    OHLCV_df = extract_stock_data(stock_name, start_date)
    # VWAP modified to use Annual cummulative sum
    OHLCV_df['typical_price'] = (OHLCV_df['High']+OHLCV_df['Low']+OHLCV_df['Close'])/3
    OHLCV_df['vw_typical_price'] = OHLCV_df['typical_price']*OHLCV_df['Volume']
    OHLCV_df['vwap_annual'] = OHLCV_df.groupby(OHLCV_df.index.year)['vw_typical_price'].cumsum() / OHLCV_df.groupby(OHLCV_df.index.year)['Volume'].cumsum()
    # Daily Returns
    OHLCV_df['daily_returns'] = OHLCV_df['Close'].pct_change()
    # 21Day Volatility
    OHLCV_df['volatility_21'] = OHLCV_df['Close'].rolling(window=21).std()
    # Simple Moving Avg 21Days & 50Days
    OHLCV_df['sma_21'] = OHLCV_df.ta.sma(length=21)
    OHLCV_df['sma_50'] = OHLCV_df.ta.sma(length=50)
    # Exponential Moving Avg 21Days & 50Days
    OHLCV_df['ema_21'] = OHLCV_df.ta.ema(length=21)
    OHLCV_df['ema_50'] = OHLCV_df.ta.ema(length=50)
    # RSI Momemtum Indicator 14Days
    rsi_df = OHLCV_df.ta.rsi()
    # MACD Momentum Indicator
    macd_df = OHLCV_df.ta.macd()
    # Bolinger Bands %B Indicator
    bbands_df = OHLCV_df.ta.bbands()
    OHLCV_append_df = pd.concat([OHLCV_df,rsi_df,macd_df,bbands_df],axis=1)    
    return(OHLCV_append_df)


In [11]:
# Predicting Stock Close & Volume-Weighted Average Price(VWAP) using Linear Models

def predict_close_vwap():
    # Next Day's "Close" as target
    predictors_df = technical_indicators()
    predictors_df = predictors_df.drop(columns = ['Open','High','Low','Adj Close','Volume',
                   'typical_price','vw_typical_price']).dropna()
    target_close_df = predictors_df[['Close']].shift(-1)
    target_vwap_df = predictors_df[['vwap_annual']].shift(-1)

    # Training Data of 11months and Testing Data of 22months alternatively 
    # with latest 33months of testing data
    train_range_months = 11
    test_range_months = 22
    split_range = list(range(len(predictors_df)//21//(train_range_months+test_range_months)))
    predictors_train = pd.DataFrame()
    
    for i in split_range:
        train_df = predictors_df.iloc[21*33*i:(21*33*i)+(21*(33-test_range_months)),:]
        predictors_train = pd.concat([predictors_train,train_df],axis=0)
        
    predictors_test = predictors_df.drop(index = predictors_train.index)
    target_train_close = target_close_df.drop(index = predictors_test.index)
    target_test_close = target_close_df.drop(index = predictors_train.index)
    target_train_vwap = target_vwap_df.drop(index = predictors_test.index)
    target_test_vwap = target_vwap_df.drop(index = predictors_train.index)
    
    #Using StandardScaler to fit predictors train data and transform both train and test data 
    predictors_scaled = scaler.fit(predictors_train)
    predictors_train_scaled = predictors_scaled.transform(predictors_train)
    predictors_test_scaled = predictors_scaled.transform(predictors_test)

    model = BayesianRidge()
    target_predictions_close_df = target_test_close.copy()
    model.fit(predictors_train_scaled,target_train_close)
    model_prediction_close = model.predict(predictors_test_scaled)
    target_predictions_close_df['Bayesian_Close'] = model_prediction_close

    target_predictions_vwap_df = target_test_vwap.copy()
    model.fit(predictors_train_scaled,target_train_vwap)
    model_prediction_vwap = model.predict(predictors_test_scaled)
    target_predictions_vwap_df['Bayesian_vwap_annual'] = model_prediction_vwap

    target_predictions_df = pd.concat([target_predictions_close_df,target_predictions_vwap_df],axis=1)

    return(target_predictions_df)



In [12]:
# Calculating Root_Mean_Square_Error(RMSE) 
# and Mean_Absolute_Percentage_Error(MAPE)

def linear_model_predictions():
    target_predictions_df = predict_close_vwap()
    dict_close = {}
    dict_vwap = {}
    rmse_close = np.sqrt(np.mean(
            (target_predictions_df.loc[:today,'Close']-target_predictions_df.loc[:today,'Bayesian_Close'])**2))
    mape_close = np.mean(np.abs(
            (target_predictions_df.loc[:today,'Close']-target_predictions_df.loc[:today,'Bayesian_Close'])/target_predictions_df.loc[:today,'Close']))*100
    dict_close['Bayesian'] = [rmse_close,mape_close]
    rmse_vwap = np.sqrt(np.mean(
            (target_predictions_df.loc[:today,'vwap_annual']-target_predictions_df.loc[:today,'Bayesian_vwap_annual'])**2))
    mape_vwap = np.mean(np.abs(
            (target_predictions_df.loc[:today,'vwap_annual']-target_predictions_df.loc[:today,'Bayesian_vwap_annual'])/target_predictions_df.loc[:today,'vwap_annual']))*100
    dict_vwap['Bayesian_vwap_annual'] = [rmse_vwap,mape_vwap]
    error_measures_close = pd.DataFrame.from_dict(dict_close, orient = 'index', columns = ['RMSE_close','MAPE_close'])
    error_measures_vwap = pd.DataFrame.from_dict(dict_vwap, orient = 'index', columns = ['RMSE_vwap','MAPE_vwap'])
    
    predictions_output = target_predictions_df[['Close','vwap_annual','Bayesian_Close','Bayesian_vwap_annual']]
    
    nxt_day = predictions_output.index[-1]+pd.Timedelta(days=1)
    #Checking for Federal Holidays
    cal = USFederalHolidayCalendar()
    holidays = pd.to_datetime(cal.holidays(today, '2024-12-31'))
    hol = list(holidays)
    for i in hol:
        if nxt_day==i:
            nxt_day = nxt_day+pd.Timedelta(days=1)
    #Checking for Weekends
    if nxt_day.weekday()==5:
        nxt_day = nxt_day+pd.Timedelta(days=2)
    elif nxt_day.weekday()==6:
        nxt_day = nxt_day+pd.Timedelta(days=1)
    else:
        nxt_day = nxt_day
    
    predictions_output.loc[nxt_day] = np.nan
    predictions_output = predictions_output.shift()
    predictions_output['Close']=predictions_output['Close'].fillna(predictions_output['Bayesian_Close'])
    predictions_output['vwap_annual']=predictions_output['vwap_annual'].fillna(predictions_output['Bayesian_vwap_annual'])

    nxt_day_predictions = predictions_output[['Close','vwap_annual']]
    model_evaluation = error_measures_close
    #display(predictions_output[['Close',best_estimate_close]].loc['2024':today].hvplot(frame_width=500,frame_height=300))
    
    return (model_evaluation,nxt_day_predictions)


In [13]:
# Feature Engineering _ for future prediction using stock_predictions as input
# OBV & ADX dropped to limit predicted features

def feature_engineering(df):
    # Daily Returns
    df['daily_returns'] = df['Close'].pct_change()
    # 21Day Volatility
    df['volatility_21'] = df['Close'].rolling(window=21).std()
    # Simple Moving Avg 21Days & 50Days
    df['sma_21'] = df['Close'].rolling(window=21).mean()
    df['sma_50'] = df['Close'].rolling(window=50).mean()
    # Exponential Moving Avg 21Days & 50Days
    df['ema_21'] = df['Close'].ewm(span=21, adjust=False).mean()
    df['ema_50'] = df['Close'].ewm(span=50, adjust=False).mean()
    # RSI Momemtum Indicator 14Days
    rsi_df = df.ta.rsi()
    # MACD Indicator
    macd_df = df.ta.macd()
    # Bolinger Bands %B Indicator
    bbands_df = df.ta.bbands()
    # Dropping trivial Columns
    append_df = pd.concat([df,rsi_df,macd_df,bbands_df],axis=1).dropna()
    # trim_df = df[['Close','vwap_annual']]
    return(append_df)


In [14]:
# Creating instance of best fit model _ BayesianRidge()
# By fitting the historical data

def best_fit_model_future_prediction(df):
    # Next Day's "Close" as target
    predictors_df = technical_indicators()
    predictors_df = predictors_df.drop(columns = ['Open','High','Low','Adj Close','Volume',
                   'typical_price','vw_typical_price']).dropna()
    target_close_df = predictors_df[['Close']].shift(-1)
    target_vwap_df = predictors_df[['vwap_annual']].shift(-1)

    predictors_future_df = feature_engineering(df)
    target_future_close_df = predictors_future_df[['Close']].shift(-1)
    target_future_vwap_df = predictors_future_df[['vwap_annual']].shift(-1)

    # Training Data of 27months and Testing Data of 6months alternatively 
    # with latest 33months of testing data
    train_range_months = 11
    test_range_months = 22
    split_range = list(range(len(predictors_df)//21//(train_range_months+test_range_months)))
    predictors_train = pd.DataFrame()
    
    for i in split_range:
        train_df = predictors_df.iloc[21*33*i:(21*33*i)+(21*(33-test_range_months)),:]
        predictors_train = pd.concat([predictors_train,train_df],axis=0)
        
    predictors_test = predictors_df.drop(index = predictors_train.index)
    target_train_close = target_close_df.drop(index = predictors_test.index)
    target_test_close = target_close_df.drop(index = predictors_train.index)
    target_train_vwap = target_vwap_df.drop(index = predictors_test.index)
    target_test_vwap = target_vwap_df.drop(index = predictors_train.index)
    
    #Using StandardScaler to fit predictors train data and transform both train and test data 
    predictors_scaled = scaler.fit(predictors_train)
    predictors_train_scaled = predictors_scaled.transform(predictors_train)
    predictors_future_scaled = predictors_scaled.transform(predictors_future_df)

    best_model = BayesianRidge()
    target_future_close = target_future_close_df.copy()
    model_instance_close = best_model
    model_instance_close.fit(predictors_train_scaled,target_train_close)
    model_prediction_close = model_instance_close.predict(predictors_future_scaled)
    target_future_close[f'{best_model}_Close'] = model_prediction_close[-1]

    target_future_vwap = target_future_vwap_df.copy()
    model_instance_vwap = best_model
    model_instance_vwap.fit(predictors_train_scaled,target_train_vwap)
    model_prediction_vwap = model_instance_vwap.predict(predictors_future_scaled)
    target_future_vwap[f'{best_model}_vwap_annual'] = model_prediction_vwap[-1]

    target_future_df = pd.concat([target_future_close,target_future_vwap],axis=1)

    #display(target_future_df.tail(1))

    nxt_day = target_future_df.index[-1]+pd.Timedelta(days=1)
    #Checking for Federal Holidays
    cal = USFederalHolidayCalendar()
    holidays = pd.to_datetime(cal.holidays(today, '2024-12-31'))
    hol = list(holidays)
    for i in hol:
        if nxt_day==i:
            nxt_day = nxt_day+pd.Timedelta(days=1)
    #Checking for Weekends
    if nxt_day.weekday()==5:
        nxt_day = nxt_day+pd.Timedelta(days=2)
    elif nxt_day.weekday()==6:
        nxt_day = nxt_day+pd.Timedelta(days=1)
    else:
        nxt_day = nxt_day
    
    target_future_df.loc[nxt_day] = np.nan
    target_future_df = target_future_df.shift()
    target_future_df['Close']=target_future_df['Close'].fillna(target_future_df[f'{best_model}_Close'])
    target_future_df['vwap_annual']=target_future_df['vwap_annual'].fillna(target_future_df[f'{best_model}_vwap_annual'])

    future_predictions = target_future_df[['Close','vwap_annual']]
    return (future_predictions)


In [15]:
# Scrapping for List of stocks in S&P500 for Linear Model ML Prediction Analysis
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
  
wiki_data = pd.read_html(url)

ticker_df = wiki_data[0]
#BRK.B & BF.B DROPPED AS DATA NOT REPORTED THROUGH YFINANCE
ticker_df = ticker_df.drop(index=[62,78]).reset_index()
ticker_df['Date added']=pd.to_datetime(ticker_df['Date added'])
ticker_df = ticker_df.sort_values(by=['Date added']).reset_index()
ticker_df = ticker_df.drop(columns=['index','level_0'])
display(ticker_df.head())
display(ticker_df.tail())

# Pick Stock from the list for analysis
ticker_list= ticker_df['Symbol'].to_list()
stock_list = ticker_df['Security'].to_list()
print(stock_list)



,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,GD,General Dynamics,Industrials,Aerospace & Defense,"Falls Church, Virginia",1957-03-04,40533,1899
2,PPG,PPG Industries,Materials,Specialty Chemicals,"Pittsburgh, Pennsylvania",1957-03-04,79879,1883
3,UNP,Union Pacific Corporation,Industrials,Rail Transportation,"Omaha, Nebraska",1957-03-04,100885,1862
4,F,Ford Motor Company,Consumer Discretionary,Automobile Manufacturers,"Dearborn, Michigan",1957-03-04,37996,1903


,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
496,UBER,Uber,Industrials,Passenger Ground Transportation,"San Francisco, California",2023-12-18,1543151,2009
497,BLDR,Builders FirstSource,Industrials,Building Products,"Irving, Texas",2023-12-18,1316835,1998
498,JBL,Jabil,Information Technology,Electronic Manufacturing Services,"St. Petersburg, Florida",2023-12-18,898293,1966
499,DECK,Deckers Brands,Consumer Discretionary,Footwear,"Goleta, California",2024-03-01,910521,1973
500,SMCI,Supermicro,Information Technology,"Technology Hardware, Storage & Peripherals","San Jose, California",2024-03-18,1375365,1993


['3M', 'General Dynamics', 'PPG Industries', 'Union Pacific Corporation', 'Ford Motor Company', 'Public Service Enterprise Group', 'ExxonMobil', 'Exelon', 'RTX Corporation', 'Entergy', 'Archer-Daniels-Midland', 'Edison International', 'Eaton Corporation', 'DTE Energy', 'General Electric', 'S&P Global', 'John Deere', 'CVS Health', 'CSX', 'Southern Company', 'Consolidated Edison', 'ConocoPhillips', 'Colgate-Palmolive', 'Coca-Cola Company (The)', 'CMS Energy', 'Chevron Corporation', 'Caterpillar Inc.', 'Boeing', 'Campbell Soup Company', 'Schlumberger', 'General Mills', 'Procter & Gamble', 'Halliburton', 'Merck & Co.', 'Abbott', 'Motorola Solutions', 'Marathon Oil', 'Lockheed Martin', 'Kroger', 'Norfolk Southern Railway', 'Xcel Energy', 'Kimberly-Clark', 'Northrop Grumman', 'Occidental Petroleum', 'International Paper', 'Bristol Myers Squibb', 'Hartford (The)', "Hershey's", 'Pfizer', 'Honeywell', 'IBM', 'Altria', 'American Electric Power', 'PepsiCo', 'Sherwin-Williams', 'Emerson Electric',

In [16]:
# display predictions
master_df=pd.DataFrame()
model_dict={}


In [17]:

for stock in stock_list:
    stock_name = stock
    start_date ='2014-01-01'
    
    model,stock_predictions = linear_model_predictions()
    k=0
    model_dict['Stock']=stock_name
    model_dict['RMSE']= round(model.iloc[0,0],2)
    model_dict['MAPE']=round(model.iloc[0,1],2)
    model_df = pd.DataFrame(model_dict,index=[k])
    k=k+1
    future_stock_price = best_fit_model_future_prediction(stock_predictions)
    # Predicting Stocks on Future Dates _ Next 14 Days
    for i in list(range(1,14)):
        future_stock_price = best_fit_model_future_prediction(future_stock_price)
        
    master_df[f'{stock}'] = future_stock_price['Close']




[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


MMM:3M
MMM:3M


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


MMM:3M
MMM:3M


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


...

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


BWA:BorgWarner
BWA:BorgWarner


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['DLTR']: ReadTimeout(ReadTimeoutError("HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out. (read timeout=10)"))


DLTR:Dollar Tree


ValueError: attempt to get argmax of an empty sequence

In [18]:
master_df

,3M,General Dynamics,PPG Industries,Union Pacific Corporation,Ford Motor Company,Public Service Enterprise Group,ExxonMobil,Exelon,RTX Corporation,Entergy,...,Edwards Lifesciences,BlackRock,Chipotle Mexican Grill,Freeport-McMoRan,Marathon Petroleum,Accenture,Mosaic Company (The),TE Connectivity,Xylem Inc.,BorgWarner
Date,,,,,,,,,,,,,,,,,,,,,
2018-10-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-10-24,184.539993,173.240005,97.779999,140.449997,8.180000,56.369999,77.620003,31.754637,76.821899,85.410004,...,46.786667,378.660004,413.000000,10.700000,68.059998,153.119995,29.520000,74.150002,65.930000,31.505281
2018-10-25,184.990005,170.600006,103.709999,144.149994,8.990000,55.849998,78.309998,31.276749,78.288231,83.500000,...,47.783333,391.959991,423.880005,11.250000,69.959999,156.919998,30.000000,75.820000,66.849998,33.204224
2018-10-26,184.949997,169.860001,102.830002,142.740005,8.980000,54.380001,77.529999,31.112696,77.551918,82.430000,...,46.713333,386.929993,438.380005,11.500000,68.860001,154.449997,29.820000,74.860001,66.519997,33.494717
2018-10-29,185.789993,167.039993,102.889999,141.339996,9.280000,55.230000,77.050003,31.440800,76.205162,84.010002,...,46.406666,390.649994,440.869995,11.090000,67.230003,152.740005,29.629999,75.680000,65.699997,34.823944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-12,110.546336,283.017122,140.877626,250.942708,12.764861,65.348714,111.157952,36.123807,95.710921,109.792385,...,91.148216,881.717841,2897.044581,46.654672,201.568476,317.856484,31.302417,145.909095,133.383667,37.322109
2024-04-15,111.241413,283.175523,140.629970,251.573612,12.737801,65.597371,110.871960,36.188615,96.134647,110.260833,...,90.798074,885.206691,2897.111529,46.709933,201.718930,316.491809,31.251121,146.138220,133.689774,37.626869
2024-04-16,111.973575,283.435800,140.401691,252.179103,12.714284,65.783557,110.584342,36.381330,96.356155,110.703955,...,90.427726,888.992839,2897.568293,46.764489,201.935827,315.351193,31.205406,146.469309,133.980585,37.942510


In [ ]:
model

## Predicting Stock Price for future dates

In [ ]:
#Predicting Day1
future_stock_price = best_fit_model_future_prediction(stock_predictions)
# Predicting Stocks on Future Dates _ Next 51 Days
for i in list(range(1,14)):
    future_stock_price = best_fit_model_future_prediction(future_stock_price)
    
#Display Future Stock Price Predictions
display(future_stock_price.loc[today:,:])

future_plot = future_stock_price.loc[today:,'Close'].hvplot(frame_width=500,frame_height=300)

display(future_plot)
display(model_plot*future_plot)

## Predicting Using Regressor Models

In [ ]:
# Single Target Regressor Models
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor

svr_rbf = SVR(kernel='rbf',C=100,gamma='auto')
# svr_poly = SVR(kernel='poly',C=100,degree=2, gamma='auto')

knn_uniform = KNeighborsRegressor(n_neighbors=2,weights='uniform')
knn_distance = KNeighborsRegressor(n_neighbors=5,weights='distance')

dtree = DecisionTreeRegressor(max_depth=10)

ens = MultiOutputRegressor(GradientBoostingRegressor(random_state=0))

# mlp = MLPRegressor(random_state=1, max_iter=100)

In [ ]:
# Predicting Stock Close & Volume-Weighted Average Price(VWAP) using Linear Models

def predict_regressor_close_vwap():
    # Next Day's "Close" as target
    predictors_df = technical_indicators()
    target_close_df = predictors_df[['Close']].shift(-1)
    target_vwap_df = predictors_df[['vwap_annual']].shift(-1)

    # Training Data of 27months and Testing Data of 6months alternatively 
    # with latest 33months of testing data
    train_range_months = 27
    test_range_months = 6
    split_range = list(range(len(predictors_df)//21//(train_range_months+test_range_months)))
    predictors_train = pd.DataFrame()
    
    for i in split_range:
        train_df = predictors_df.iloc[21*33*i:(21*33*i)+(21*(33-test_range_months)),:]
        predictors_train = pd.concat([predictors_train,train_df],axis=0)
        
    predictors_test = predictors_df.drop(index = predictors_train.index)
    target_train_close = target_close_df.drop(index = predictors_test.index)
    target_test_close = target_close_df.drop(index = predictors_train.index)
    target_train_vwap = target_vwap_df.drop(index = predictors_test.index)
    target_test_vwap = target_vwap_df.drop(index = predictors_train.index)
    
    #Using StandardScaler to fit predictors train data and transform both train and test data 
    predictors_scaled = scaler.fit(predictors_train)
    predictors_train_scaled = predictors_scaled.transform(predictors_train)
    predictors_test_scaled = predictors_scaled.transform(predictors_test)

    list_of_regressors = [svr_rbf,knn_uniform,knn_distance,dtree,ens]
    target_predictions_close_df = target_test_close.copy()
    for model in list_of_regressors:
        model_instance_close = model
        model_instance_close.fit(predictors_train_scaled,target_train_close)
        model_prediction_close = model_instance_close.predict(predictors_test_scaled)
        target_predictions_close_df[f'{model}_Close'] = model_prediction_close

    target_predictions_vwap_df = target_test_vwap.copy()
    for model in list_of_regressors:
        model_instance_vwap = model
        model_instance_vwap.fit(predictors_train_scaled,target_train_vwap)
        model_prediction_vwap = model_instance_vwap.predict(predictors_test_scaled)
        target_predictions_vwap_df[f'{model}_vwap_annual'] = model_prediction_vwap

    target_predictions_df = pd.concat([target_predictions_close_df,target_predictions_vwap_df],axis=1)

    display(target_predictions_df.tail())
    display(target_predictions_close_df.hvplot(frame_width=500,frame_height=300))
    return(target_predictions_close_df,target_predictions_vwap_df,target_predictions_df)


In [ ]:
# Calculating Root_Mean_Square_Error(RMSE) 
# and Mean_Absolute_Percentage_Error(MAPE)

def regressor_model_predictions():
    display(extract_stock_data(stock_name, start_date).head(3))
    display(extract_stock_data(stock_name, start_date).tail(3))
    display(technical_indicators().tail())
    today = datetime.today().strftime('%Y-%m-%d')
    target_close_predictions_df,target_vwap_predictions_df,target_predictions_df = predict_regressor_close_vwap()
    column_range_close = list(target_close_predictions_df.columns)
    column_range_vwap = list(target_vwap_predictions_df.columns)
    dict_close = {}
    dict_vwap = {}
    for i in column_range_close[1:]:
        rmse_close = np.sqrt(np.mean(
            (target_predictions_df.loc[:today,'Close']-target_predictions_df.loc[:today,i])**2))
        mape_close = np.mean(np.abs(
            (target_predictions_df.loc[:today,'Close']-target_predictions_df.loc[:today,i])/target_predictions_df.loc[:today,'Close']))*100
        dict_close[f'{i}'] = [rmse_close,mape_close]
    for i in column_range_vwap[1:]:  
        rmse_vwap = np.sqrt(np.mean(
            (target_predictions_df.loc[:today,'vwap_annual']-target_predictions_df.loc[:today,i])**2))
        mape_vwap = np.mean(np.abs(
            (target_predictions_df.loc[:today,'vwap_annual']-target_predictions_df.loc[:today,i])/target_predictions_df.loc[:today,'vwap_annual']))*100
        dict_vwap[f'{i}'] = [rmse_vwap,mape_vwap]
    error_measures_close = pd.DataFrame.from_dict(dict_close, orient = 'index', columns = ['RMSE_close','MAPE_close'])
    error_measures_vwap = pd.DataFrame.from_dict(dict_vwap, orient = 'index', columns = ['RMSE_vwap','MAPE_vwap'])
    error_measures_close = error_measures_close.sort_values(by=['MAPE_close'])
    error_measures_vwap = error_measures_vwap.sort_values(by=['MAPE_vwap'])

    display(error_measures_close)
    display(error_measures_close.hvplot.bar(rot=90,frame_width=700,frame_height=300))

    best_estimate_close = error_measures_close.index[0]
    best_estimate_vwap = error_measures_vwap.index[0]

    predictions_output = target_predictions_df[['Close','vwap_annual',best_estimate_close, best_estimate_vwap]]
    
    nxt_day = predictions_output.index[-1]+pd.Timedelta(days=1)
    #Checking for Federal Holidays
    cal = USFederalHolidayCalendar()
    holidays = pd.to_datetime(cal.holidays(today, '2024-12-31'))
    hol = list(holidays)
    for i in hol:
        if nxt_day==i:
            nxt_day = nxt_day+pd.Timedelta(days=1)
    #Checking for Weekends
    if nxt_day.weekday()==5:
        nxt_day = nxt_day+pd.Timedelta(days=2)
    elif nxt_day.weekday()==6:
        nxt_day = nxt_day+pd.Timedelta(days=1)
    else:
        nxt_day = nxt_day
    
    predictions_output.loc[nxt_day] = np.nan
    predictions_output = predictions_output.shift()
    predictions_output['Close']=predictions_output['Close'].fillna(predictions_output[best_estimate_close])
    predictions_output['vwap_annual']=predictions_output['vwap_annual'].fillna(predictions_output[best_estimate_vwap])

    predictions = predictions_output[['Close','vwap_annual']]
    best_model = best_estimate_close.removesuffix('_Close')
    comparison_plot = predictions_output[['Close',best_estimate_close]].loc['2024':today].hvplot(frame_width=300,frame_height=300)
    
    return (comparison_plot,best_model,predictions)



In [ ]:
regressor_plot,regressor,stock_predictions_regr = regressor_model_predictions()

display(stock_predictions_regr.tail())
print(f'Best fit Regressor Model: {regressor}')
regressor_plot


In [ ]:
# Creating instance of best fit regressor model
# By fitting the historical data

def best_fit_regr_future_prediction(df):
    # Next Day's "Close" as target
    predictors_df = technical_indicators()
    predictors_df = predictors_df.drop(columns=['Volume','OBV_in_million','adx_14'])
    target_close_df = predictors_df[['Close']].shift(-1)
    target_vwap_df = predictors_df[['vwap_annual']].shift(-1)

    predictors_future_df = feature_engineering(df)
    target_future_close_df = predictors_future_df[['Close']].shift(-1)
    target_future_vwap_df = predictors_future_df[['vwap_annual']].shift(-1)

    # Training Data of 27months and Testing Data of 6months alternatively 
    # with latest 33months of testing data
    train_range_months = 27
    test_range_months = 6
    split_range = list(range(len(predictors_df)//21//(train_range_months+test_range_months)))
    predictors_train = pd.DataFrame()
    
    for i in split_range:
        train_df = predictors_df.iloc[21*33*i:(21*33*i)+(21*(33-test_range_months)),:]
        predictors_train = pd.concat([predictors_train,train_df],axis=0)
        
    predictors_test = predictors_df.drop(index = predictors_train.index)
    target_train_close = target_close_df.drop(index = predictors_test.index)
    target_test_close = target_close_df.drop(index = predictors_train.index)
    target_train_vwap = target_vwap_df.drop(index = predictors_test.index)
    target_test_vwap = target_vwap_df.drop(index = predictors_train.index)
    
    #Using StandardScaler to fit predictors train data and transform both train and test data 
    predictors_scaled = scaler.fit(predictors_train)
    predictors_train_scaled = predictors_scaled.transform(predictors_train)
    predictors_future_scaled = predictors_scaled.transform(predictors_future_df)

    best_model_close = ens
    target_future_close = target_future_close_df.copy()
    model_instance_close = best_model_close
    model_instance_close.fit(predictors_train_scaled,target_train_close)
    model_prediction_close = model_instance_close.predict(predictors_future_scaled)
    target_future_close[f'{best_model_close}_Close'] = model_prediction_close

    best_model_vwap = ens
    target_future_vwap = target_future_vwap_df.copy()
    model_instance_vwap = best_model_vwap
    model_instance_vwap.fit(predictors_train_scaled,target_train_vwap)
    model_prediction_vwap = model_instance_vwap.predict(predictors_future_scaled)
    target_future_vwap[f'{best_model_vwap}_vwap_annual'] = model_prediction_vwap

    target_future_df = pd.concat([target_future_close,target_future_vwap],axis=1)

    display(target_future_df.tail(1))

    nxt_day = target_future_df.index[-1]+pd.Timedelta(days=1)
    #Checking for Federal Holidays
    cal = USFederalHolidayCalendar()
    holidays = pd.to_datetime(cal.holidays(today, '2024-12-31'))
    hol = list(holidays)
    for i in hol:
        if nxt_day==i:
            nxt_day = nxt_day+pd.Timedelta(days=1)
    #Checking for Weekends
    if nxt_day.weekday()==5:
        nxt_day = nxt_day+pd.Timedelta(days=2)
    elif nxt_day.weekday()==6:
        nxt_day = nxt_day+pd.Timedelta(days=1)
    else:
        nxt_day = nxt_day
    
    target_future_df.loc[nxt_day] = np.nan
    target_future_df = target_future_df.shift()
    target_future_df['Close']=target_future_df['Close'].fillna(target_future_df[f'{best_model_close}_Close'])
    target_future_df['vwap_annual']=target_future_df['vwap_annual'].fillna(target_future_df[f'{best_model_vwap}_vwap_annual'])

    predictions = target_future_df[['Close','vwap_annual']]
    
    return (predictions)


In [ ]:
#Predicting Day1
future_stock_price_regr = best_fit_regr_future_prediction(stock_predictions_regr)
# Predicting Stocks on Future Dates _ Next 51 Days
for i in list(range(1,14)):
    future_stock_price_regr = best_fit_regr_future_prediction(future_stock_price_regr)

#Display Regressor Model Predictions
display(future_stock_price_regr.loc[today:,:])

future_plot_regr = future_stock_price_regr.loc[today:,'Close'].hvplot(frame_width=300,frame_height=300)

display(future_plot_regr)
display(regressor_plot*future_plot_regr)

In [ ]:
regressor_plot*future_plot*future_plot_regr

In [ ]:
linear_close = future_stock_price[['Close']].rename(columns={'Close':'Close_Linear'})
regr_close = future_stock_price_regr[['Close']].rename(columns={'Close':'Close_Regr'})

combined_df = pd.concat([linear_close,regr_close],axis=1).dropna()

display(combined_df.loc[today:])

combined_df.loc[today:].hvplot()


In [ ]:

for i in list(range(1,8)):
    future_stock_price_regr=best_fit_regr_future_prediction(future_stock_price_regr)
    future_stock_price_regr



In [ ]:
future_stock_price_regr = future_stock_price_regr[['Close','vwap_annual']].dropna()

future_stock_price_regr